In [2]:
from instruction import keypair_from_json
from processor import mint_nft
from solana.keypair import Keypair
from solana.rpc.api import Client
from state import ClassEnum
client = Client("https://api.devnet.solana.com")
print("Client is connected" if client.is_connected() else "Client is Disconnected")
payer_keypair = keypair_from_json("./keypair.json")
print(payer_keypair.public_key)
mint_keypair = Keypair()
mint_nft(payer_keypair, mint_keypair, ClassEnum.enum.Benitoite(), client)

Client is connected
5sJaXHGsEa3VF1h2RR326mdTQkbtww6qnmNuGTJoNv88
9AuYZkf1uHWYDZ9S4aUeBY2D4AhsJeWd9j5HvVks7417


{'jsonrpc': '2.0',
 'result': '3WemmmAE4CUd4B3XomXpHT4mVJMUDkjvZyipmaz4rktwfZMxoeNSPidwdAYJHCnvtHYd14UdtrQA67P1iubHr83s',
 'id': 2}